# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, linked_variables: Dict[str, str] = {}) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negat

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables={})

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': {}}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables": {"a": [0.0, 1000.0], "b": [-1.0, 1.0]}, "constraints": {"e": ["LESS_THAN", 2.0], "f": ["GREATER_THAN", 0.0]}, "objectives": {"c": "MAXIMIZE", "d": "MINIMIZE"}, "constants": {"g": 1234}, "linked_variables": {}}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,498.438047,-0.918645,1234,497.519402,499.356692,995.038805,998.713383
6,740.391639,-0.033625,1234,740.358015,740.425264,1480.716029,1480.850528
7,179.090795,-0.268252,1234,178.822543,179.359048,357.645086,358.718096
8,421.780674,0.457940,1234,422.238613,421.322734,844.477226,842.645468
9,829.930330,-0.530353,1234,829.399977,830.460683,1658.799955,1660.921367
10,849.557640,0.978719,1234,850.536358,848.578921,1701.072717,1697.157842
11,954.366189,0.980850,1234,955.347039,953.385339,1910.694079,1906.770678
12,599.032540,0.412657,1234,599.445197,598.619883,1198.890394,1197.239767
13,155.863631,-0.426488,1234,155.437143,156.290120,310.874286,312.580240
14,777.437731,-0.128871,1234,777.308860,777.566602,1554.617720,1555.133203


In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-497.519402,499.356692
6,-740.358015,740.425264
7,-178.822543,179.359048
8,-422.238613,421.322734
9,-829.399977,830.460683
10,-850.536358,848.578921
11,-955.347039,953.385339
12,-599.445197,598.619883
13,-155.437143,156.290120
14,-777.308860,777.566602


In [13]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-497.519402,499.356692
6,-740.358015,740.425264
7,-178.822543,179.359048
8,-422.238613,421.322734
9,-829.399977,830.460683
10,-850.536358,848.578921
11,-955.347039,953.385339
12,-599.445197,598.619883
13,-155.437143,156.290120
14,-777.308860,777.566602


In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-497.51940227,  499.35669174],
       [-740.35801471,  740.42526389],
       [-178.82254294,  179.35904778],
       [-422.23861321,  421.3227338 ],
       [-829.39997726,  830.46068339],
       [-850.53635833,  848.57892076],
       [-955.34703932,  953.38533919],
       [-599.4451968 ,  598.61988325],
       [-155.43714297,  156.29011993],
       [-777.30885987,  777.56660166]])

In [15]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,993.038805,-998.713383
6,1478.716029,-1480.850528
7,355.645086,-358.718096
8,842.477226,-842.645468
9,1656.799955,-1660.921367
10,1699.072717,-1697.157842
11,1908.694079,-1906.770678
12,1196.890394,-1197.239767
13,308.874286,-312.580240
14,1552.617720,-1555.133203


In [16]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
